In [5]:
from datasets import Dataset, DatasetDict
import pandas as pd
from huggingface_hub import notebook_login

In [6]:
train_data = pd.read_csv('data/train_data.csv')
test_data = pd.read_csv('data/test_data.csv')
valid_data = pd.read_csv('data/validation_data.csv')

# Convert to Hugging Face dataset
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
valid_dataset = Dataset.from_pandas(valid_data)

# Create a dataset dictionary
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': valid_dataset
})

In [7]:
notebook_login()

dataset_dict.push_to_hub("techdebt")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/208 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/70 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("huggingface/CodeBERTa-small-v1")
techdebt = load_dataset("davidgaofc/techdebt")


In [5]:
def preprocess_function(examples):
    return tokenizer(examples["Diff"], truncation=True)

tokenized_techdebt = techdebt.map(preprocess_function, batched=True)

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def format_dataset(examples):
    examples['label'] = examples['FaultInducingLabel']
    return examples

tokenized_techdebt = tokenized_techdebt.map(format_dataset)

Map:   0%|          | 0/69155 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:   0%|          | 0/69155 [00:00<?, ? examples/s]

In [6]:
import torch
x = torch.rand(5, 3)
x

tensor([[0.8047, 0.5704, 0.8099],
        [0.7021, 0.0607, 0.5892],
        [0.3870, 0.0667, 0.8759],
        [0.8900, 0.6159, 0.9113],
        [0.4112, 0.6646, 0.0825]])

In [7]:

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("huggingface/CodeBERTa-small-v1", num_labels=2)

training_args = TrainingArguments(output_dir="training", learning_rate=2e-5, weight_decay=0.01, num_train_epochs=3)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_techdebt["train"],
    eval_dataset=tokenized_techdebt["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

temp = trainer.evaluate()
import datetime

model.save_pretrained("model_checkpoints/techdebt_model"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tokenizer.save_pretrained("model_checkpoints/techdebt_tokenizer"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at huggingface/CodeBERTa-small-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


RuntimeError: MPS backend out of memory (MPS allocated: 1.32 GB, other allocations: 16.75 GB, max allowed: 18.13 GB). Tried to allocate 152.34 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).